In [ ]:
!pip install pathway bokeh --quiet

In [ ]:
# importing necessary Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [ ]:
# importing the csv file from GitHub
url='https://raw.githubusercontent.com/rishmish0806/Capstone/refs/heads/main/dataset.csv'
df=pd.read_csv(url)

In [ ]:
# creating 14 DataFrames for each system
df.SystemCodeNumber.unique()

df_locations=[df[df['SystemCodeNumber']==loc] for loc in df.SystemCodeNumber.unique()]
num_locs=len(df_locations)

In [ ]:
# creating TimeStamp feature in every DataSet
for df_i in df_locations:
  # Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
  df_i['Timestamp'] = pd.to_datetime(df_i['LastUpdatedDate'] + ' ' + df_i['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

  # Sort the DataFrame by the new 'Timestamp' column and reset the index
  df_i= df_i.sort_values('Timestamp').reset_index(drop=True)

In [ ]:
idx=0
for df_i in df_locations:
  df_temp=df_i[["Timestamp", "Occupancy", "Capacity"]]
  df_temp['Occupancy_ratio']=df_i['Occupancy']/df_i['Capacity']
  df_temp['Price']=df_temp['Occupancy_ratio']
  df_temp['Price'] = 10 + df_temp['Occupancy_ratio'].cumsum()

  df_temp=df_temp[['Timestamp','Price']]
  df_temp.to_csv(f"parking_stream{idx}.csv", index=False)
  idx+=1

In [ ]:
# creating a Parking-Stream Schema
class ParkingSchema(pw.Schema):
    Timestamp: str   # Timestamp of the observation (should ideally be in ISO format)
    Price:np.double
    Occupancy:int
    Capacity:int

In [ ]:
# creating mock-stream for different locations
data_locations=[pw.demo.replay_csv(f"parking_stream{i}.csv", schema=ParkingSchema, input_rate=1000) for i in range(num_locs)]



In [ ]:
# creating date_time Format

fmt = "%Y-%m-%d %H:%M:%S"


# creating mock_stream with time and day columns
data_with_time_locations= [data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00"),
) for data in data_locations
]

# type(data_with_time_locations[0])

In [ ]:
delta_window_locations = [
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    # Use select() instead of reduce() to keep original values
    .select(
        t=pw.this._pw_window_end,
        price=pw.this.Price  # Keep original price values
    )
    for data_with_time in data_with_time_locations
]

In [ ]:
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

In [ ]:
# creating a dash_board


# Creating a list to store visualizations
plots = []

# Generate a plot for each data source
for i, source in enumerate(delta_window_locations):
    # Create visualization using your existing price_plotter
    viz = source.plot(price_plotter, sorting_col="t")
    plots.append(viz)

# Combine all plots into a Column (vertical layout)
dashboard = pn.Column(*plots, sizing_mode="stretch_width")

# Serve the dashboard
dashboard.servable("Multi-Plot Dashboard")


In [ ]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()